# Regresión Logística Binaria

<img src="img/Log_neuron_1.png" width="600">

<img src="img/Log_CrossEntropy_2.png" width="700">

<img src="img/MiniBatch_GD_3.png" width="700">

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn, optim
import torch.nn.functional as F

import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
##########################
### CONFIGURACION
##########################

RANDOM_SEED = 1
BATCH_SIZE = 64
NUM_EPOCHS = 500
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
lr = 0.01

torch.manual_seed(RANDOM_SEED)

In [3]:
breast = datasets.load_breast_cancer()
X = breast['data']
y = breast['target']

X = (X - X.mean(axis=0))/X.std(axis=0)
# sc = StandardScaler()
# X = sc.fit_transform(X)

In [4]:
print(X.shape)
print(y.shape)

(569, 30)
(569,)


In [5]:
X_train,X_valid,y_train,y_valid = train_test_split(X, y, test_size=0.4, random_state=RANDOM_SEED)

In [6]:
X_train, y_train, X_valid, y_valid = map( lambda x: torch.tensor(x, dtype=torch.float32),
                                         (X_train, y_train, X_valid, y_valid) )

In [7]:
train_ds = TensorDataset(X_train, y_train)
valid_ds = TensorDataset(X_valid, y_valid)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size = 2*BATCH_SIZE)

In [8]:
print(X_train.shape)
print(y_train.shape)

torch.Size([341, 30])
torch.Size([341])


### 1. Forma explícita

In [9]:
# class Logistic(nn.Module):
#     def __init__(self, num_features):
#         super().__init__()
#         self.weights = nn.Parameter(torch.zeros(num_features))
#         self.bias = nn.Parameter(torch.zeros(1))
        
#     def forward(self,x):
#         return x @ self.weights + self.bias

### 2. Forma semi-explicita

In [10]:
# class Logistic(nn.Module):
#     def __init__(self, num_features):
#         super().__init__()
#         self.lin = nn.Linear(num_features,1)
#         self.lin.weight.detach().zero_()
#         self.lin.bias.detach().zero_()
        
#     def forward(self,x):
#         return self.lin(x)

In [11]:
# model = Logistic(X.shape[1])

### 3. Secuencial

In [ ]:
model = nn.Sequential(
    nn.Linear(X.shape[1],1),
)

In [12]:
# Esto es para inicializar los pesos a ceros una vez definido el modelo
# (por defecto con nn se inicializan con Xavier, Hu o similar)
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.zeros_(m.weight)
        nn.init.zeros_(m.bias)
        print(m.weight)

model.apply(weights_init)

Parameter containing:
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]], requires_grad=True)


Sequential(
  (0): Linear(in_features=30, out_features=1, bias=True)
)

In [13]:
loss_func = F.binary_cross_entropy_with_logits

In [14]:
model = model.to(DEVICE)

opt = optim.SGD(model.parameters(),lr = lr)

In [15]:
def loss_acc_batch(x,y, acc = None):
    yp = model(x.to(DEVICE))
    yt = y.to(DEVICE)
    # .squeeze() es para volver los vectores unidimensionales y evitar problemas en las tres formas
    # de hacer el modelo, ya que el "y" original es 1D, mientras que la salida del modelo
    # es vector 2D (columna) para los casos en que involucra nn.Linear()
    loss = loss_func(yp.squeeze(), yt.squeeze())
    out = [loss, len(x)]
    
    if acc is not None:
        y_pred = (torch.sigmoid(yp) >= 0.5).float() 
        accuracy = (y_pred.squeeze() == yt.squeeze()).float().mean().item()
        out.append(accuracy)
    return out

In [16]:
def acc(out,y):
    y_pred = (torch.sigmoid(out) >= 0.5).float() 
    return (y_pred == y).float().sum()

In [17]:
def train():
    
    train_losses = []
    valid_losses = []
    for epoch in range(NUM_EPOCHS):
        model.train()
        losses = 0
        nums = 0
        for x, y in train_dl:
            loss, l = loss_acc_batch(x,y)
            loss.backward()
            opt.step()
            opt.zero_grad()
            
            losses += loss.item() * l
            nums += l
        train_loss = losses / nums
        train_losses.append(train_loss)
        
        model.eval()
        with torch.no_grad():
            losses, nums, accs = zip(*[loss_acc_batch(xb, yb, acc=True) for xb, yb in valid_dl])
        losses = [l.item() for l in losses]
        valid_loss = np.sum(np.multiply(losses,nums)) / np.sum(nums)
        valid_losses.append(valid_loss)
        
        valid_acc = np.sum(np.multiply(accs,nums)) / np.sum(nums)
        
        if epoch % 10 == 0:
            print(f"epoch: {epoch},    train_loss: {train_loss:.4f} \
            valid_loss: {valid_loss:.4f}, valid_acc: {valid_acc:.4f}")
    

In [18]:
train()

epoch: 0,    train_loss: 0.6490             valid_loss: 0.5941, valid_acc: 0.9254
epoch: 10,    train_loss: 0.2848             valid_loss: 0.3143, valid_acc: 0.9386
epoch: 20,    train_loss: 0.2123             valid_loss: 0.2479, valid_acc: 0.9430
epoch: 30,    train_loss: 0.1787             valid_loss: 0.2154, valid_acc: 0.9430
epoch: 40,    train_loss: 0.1585             valid_loss: 0.1952, valid_acc: 0.9430
epoch: 50,    train_loss: 0.1449             valid_loss: 0.1811, valid_acc: 0.9474
epoch: 60,    train_loss: 0.1350             valid_loss: 0.1708, valid_acc: 0.9474
epoch: 70,    train_loss: 0.1273             valid_loss: 0.1627, valid_acc: 0.9474
epoch: 80,    train_loss: 0.1213             valid_loss: 0.1562, valid_acc: 0.9518
epoch: 90,    train_loss: 0.1164             valid_loss: 0.1505, valid_acc: 0.9518
epoch: 100,    train_loss: 0.1122             valid_loss: 0.1459, valid_acc: 0.9561
epoch: 110,    train_loss: 0.1087             valid_loss: 0.1419, valid_acc: 0.9518
epo

### Ejercicio

- Realice la curva del Loss para entrenamiento y validación vs cada epoch
- Realice la curva del accuracy para el entrenamiento y la validación para cada epoch